# Batch Lever Press Analysis
## Notebook for preprocessing and analyzing coactivity spine data

In [ ]:
import os
import numpy as np
from Lab_Analyses.Spine_Analysis.preprocess_spine_data import organize_dual_spine_data
from Lab_Analyses.Spine_Analysis.grouped_coactivity_analysis_v2 import grouped_coactivity_analysis
from Lab_Analyses.Spine_Analysis.coactivity_plasticity_analysis import Coactivity_Plasticity
from Lab_Analyses.Utilities.save_load_pickle import load_pickle

### Datasets
Specify mice which mice to be processed and analyzed

In [ ]:
# Mouse ids
id_1 = "JW082"
id_2 = "JW083"
id_3 = "JW090"
id_4 = "JW092"

mouse_list = [id_1, id_2, id_3, id_4]

### Preprocessing
Preprocess spine datasets

In [ ]:
# Do you want to reprocess the data
reprocess = True
channels = {"GluSnFr": "GreenCh", "Calcium": "RedCh"}

# Process data
for mouse in mouse_list:
    organize_dual_spine_data(mouse, channels, redetection=True, save=True, structural=True)

### Grouped Coactivity Analysis
Perform coactivity analysis across all mice and FOVs for a given set of imaging sessions

In [ ]:
# Set up parameters
## Save path
group_path = r'C:\Users\Jake\Desktop\Analyzed_data\grouped'
experiment_dir = "Dual_Spine_Imaging"
coactivity_dir = "Coactivity_Data"
coactivity_path = os.path.join(group_path, experiment_dir, coactivity_dir)

## Analysis parameters
reanalyze = True
save = True
days = ["Early", "Middle", "Late"]
#days = ["Middle"]
followup = True
movement_epochs = [None, "movement", "rewarded", "unrewarded",
                   "nonmovement"]
#movement_epochs = ["unrewarded"]
zscore = False
volume_norm = True

coactivity_data = {}
for day in days:
    epoch_data = {}
    print(f"-{day}")
    for epoch in movement_epochs:
        print(f"--{epoch}")
        coactivity = grouped_coactivity_analysis(
            mouse_list,
            day,
            followup=followup,
            activity_window=(-2,4),
            movement_epochs=epoch,
            zscore=zscore,
            volume_norm=volume_norm,
            save=save,
            save_path=coactivity_path,
        )
        if epoch is None:
            epoch = "session"
        epoch_data[epoch] = coactivity
    coactivity_data[day] = epoch_data

### Coactivity Plasticity Analysis
Perform analysis to relate subsequent plasticity to coactivity 

In [ ]:
## Save path
group_path = r'C:\Users\Jake\Desktop\Analyzed_data\grouped'
experiment_dir = "Dual_Spine_Imaging"
plasticity_dir = "Coactivity_Plasticity"
plasticity_path = os.path.join(group_path, experiment_dir, plasticity_dir)

In [ ]:

## Analysis parameters
threshold = 0.5
exclude = "Shaft Spine"
save = True

plasticity_data = {}
for key, value in coactivity_data.items():
    sub_data = {}
    for k, v in value.items():
        plasticity = Coactivity_Plasticity(
            v, threshold, exclude, save=save, save_path=plasticity_path
        )
        sub_data[k] = plasticity
    plasticity_data[key] = sub_data

### Visualize Coactivity Plasticity Data
Plot selected data and variables from individual datasets.
Use data generated above of load in a dataset

In [ ]:
# Select data to plot
load_data = True
initial_path = r"C:\Users\Jake\Desktop\Figures\grouped_data"
figure_path = os.path.join(initial_path, experiment_dir, plasticity_dir, "session")
if not os.path.isdir(figure_path):
    os.makedirs(figure_path)
if load_data:
    load_name = "Early_session_dFoF_norm_0.5_coactivity_plasticity_data"
    plot_data = load_pickle([load_name], plasticity_path)[0]
else:
    plot_data = plasticity_data["Early"]["movement"]


In [ ]:
# Print out variable names
for key in plot_data.__dict__.keys():
    print(key)

In [ ]:
initial_path = r"C:\Users\Jake\Desktop\Analyzed_data\individual"
behavior_path = os.path.join(initial_path, "JW082", "behavior")
final_day = sorted([x[0] for x in os.walk(behavior_path)])[-1]
final_day

In [ ]:
import numpy as np
a = np.array([[1,1,1,1,1,1],
              [2,2,2,2,2,2],
              [3,3,3,3,3,3],
              [4,4,4,4,4,4]])
b = a[:, [0,1,4,5]]
np.sum(b, axis=1)

In [ ]:
# Correlation Plots
plot_data.plot_volume_correlation(
    variable_name="dend_active_frac_move",
    volume_type="relative_volume",
    CI=95,
    ytitle="Dendrite dF/F",
    xlim=None,
    ylim=None,
    face_color="mediumblue",
    edge_color="white",
    edge_width=1,
    s_alpha=0.4,
    line_color="mediumblue",
    line_width=1,
    save=False,
    save_path=figure_path
)

In [ ]:
# Group mean activity plots
plot_data.plot_group_scatter_plots(
    variable_name="local_movement_corr",
    group_type="rwd_movement_spines",
    mean_type="median",
    err_type="std",
    marker="o",
    figsize=(4,5),
    ytitle="Spine dF/F",
    ylim=None,
    s_colors=["mediumblue", "firebrick"],
    s_size=5,
    s_alpha=0.3,
    m_colors=["mediumblue", "firebrick"],
    m_size=6,
    save=False,
    save_path=figure_path,
)

In [ ]:
print(np.sum(plot_data.enlarged_spines)/len(plot_data.enlarged_spines))
print(np.sum(plot_data.shrunken_spines)/len(plot_data.enlarged_spines))
print(np.sum(plot_data.stable_spines)/len(plot_data.enlarged_spines))
e= np.sum(plot_data.enlarged_spines*plot_data.movement_spines)/np.sum(plot_data.movement_spines)
s= np.sum(plot_data.shrunken_spines*plot_data.movement_spines)/np.sum(plot_data.movement_spines)
stab = np.sum(plot_data.stable_spines*plot_data.movement_spines)/np.sum(plot_data.movement_spines)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
sns.set_style("ticks")


plt.figure(figsize=(4,5))
plt.bar(x=[0,1,2], height=[e, s, stab], color=["darkorange", "forestgreen", "silver"])
plt.ylabel("Fraction of MRSs")
plt.xticks(ticks=[0,1,2], labels=["enlarged", "shrunken", "stable"])
plt.ylim(0, 0.5)
plt.tight_layout()
fname = os.path.join(figure_path, "frac_MRSs")
plt.savefig(fname + ".pdf")

In [ ]:

plot_data.plot_multi_group_scatter_plots(
    variable_name="relative_spine_dend_move_corr",
    group_type="movement_spines",
    subgroup_type="plastic_spines",
    mean_type="median",
    err_type="std",
    marker="o",
    figsize=(5,5),
    ytitle="Relative Learned Movement Correlation (spine-dendrite)",
    ylim=None,
    s_colors=["darkorange", "forestgreen", "silver"],
    s_alpha=0.3,
    s_size=5,
    m_colors=["darkorange", "forestgreen", "silver"],
    m_size=6,
    save=True,
    save_path=figure_path,
)

In [ ]:
plot_data.plot_group_spine_mean_traces(
    group_type="plastic_spines",
    trace_type="conj_coactive_dend_traces",
    exclude=[],
    avlines=[0],
    figsize=(5,5),
    colors=["darkorange", "forestgreen", "silver"], 
    ylim=None,
    save=False,
    save_path=figure_path
)

In [ ]:
plot_data.plot_spine_coactivity_distance(
    group_type="plastic_spines",
    figsize=(5,5),
    norm=False,
    colors=["mediumblue", "firebrick", "silver"],
    ylim=None,
    save=False,
    save_path=figure_path,
)

In [ ]:
plot_data.plot_histogram(
    variable="relative_volumes",
    bins=20,
    avlines=None,
    figsize=(5,5),
    color="blue",
    alpha=0.4,
    save=True,
    save_path=figure_path
)

In [ ]:
plot_data.plot_group_mean_heatmaps(
    trace_type="rwd_move_spine_traces",
    group_type="plastic_spines",
    figsize=(10,5),
    hmap_range=(0.2,1),
    center=None,
    sorted="peak",
    normalize=True,
    cmap="plasma",
    save=False,
    save_path=figure_path
)